# **Sequential Chain**

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain, SequentialChain

In [3]:
email = """
Betreff: Unerträgliche Verzögerung bei der Zustellung meiner Kreditkarte – Drittes Monat ohne Lösung!

Sehr geehrte Damen und Herren,

ich bin fassungslos und zutiefst enttäuscht über die absolute Inkompetenz, die Ihre Bank in Bezug auf die Zustellung meiner Kreditkarte an den Tag legt. Es sind nun fast drei Monate vergangen, seitdem ich die Karte beantragt habe, und ich habe sie immer noch nicht erhalten – und das trotz mehrfacher Zusicherungen Ihrerseits.

Was zur Hölle geht bei Ihnen vor? Drei Monate Wartezeit für eine simple Kreditkarte sind ein absolutes Unding und völlig unakzeptabel! Ich erwarte nicht nur eine sofortige Erklärung für diese erschreckende Verzögerung, sondern auch eine umgehende Lösung. Ihre Bank hat es in keiner Weise geschafft, meine Erwartung an einen einfachen Service zu erfüllen. So etwas habe ich noch nie erlebt und werde es auch nicht tolerieren!

Ich fordere Sie hiermit auf, mir innerhalb der nächsten 24 Stunden eine verbindliche Antwort zu geben, in der Sie mir den genauen Lieferstatus mitteilen und die schnellstmögliche Zustellung meiner Karte garantieren. Sollte dies nicht passieren, sehe ich mich gezwungen, sämtliche Konsequenzen zu ziehen und nicht nur meine Geschäftsbeziehung zu Ihrer Bank zu beenden, sondern auch rechtliche Schritte einzuleiten.

Ich erwarte eine umgehende Rückmeldung und erwarte, dass Sie Ihre Arbeit endlich auf die Reihe bekommen.

Mit wütenden Grüßen,
"""

In [5]:
llm = ChatOpenAI(model='gpt-3.5-turbo', temperature=0)

In [7]:
# First we will detect the language of the email
template_lang = "Return the language of the email. Only written the language the email was written in:\n {email}"
prompt_lang = ChatPromptTemplate.from_template(template_lang)
chain_lang = LLMChain(llm = llm, prompt=prompt_lang, output_key='language')

In [8]:
# Translate the email into English language
template_translate = "Translate this {email} from {language} to English."
prompt_translate = ChatPromptTemplate.from_template(template_translate)
chain_translate = LLMChain(llm = llm, prompt=prompt_translate, output_key = 'translated_email')

In [9]:
# Translate the email into English language
template_summary = """Create a short summary of the email: {translated_email}. 
The translation must highlight the key issues faced by the customer"""
prompt_summary = ChatPromptTemplate.from_template(template_summary)
chain_summary = LLMChain(llm = llm, prompt=prompt_summary, output_key = 'summary')

In [10]:
seq_chain = SequentialChain(chains = [chain_lang, chain_translate, chain_summary],
                            input_variables=['email'],
                            output_variables=['language', 'translated_email', 'summary'],
                            verbose=False
                           )

In [11]:
response = seq_chain(email)

/tmp/ipykernel_139/110762567.py:1: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = seq_chain(email)


In [12]:
response.keys()

dict_keys(['email', 'language', 'translated_email', 'summary'])

In [13]:
response.get('language')

'German'

In [14]:
print(response.get('translated_email'))

Subject: Unbearable Delay in the Delivery of my Credit Card - Third Month without a Solution!

Dear Sir/Madam,

I am appalled and deeply disappointed by the absolute incompetence that your bank has shown in delivering my credit card. It has been almost three months since I applied for the card, and I still have not received it - despite multiple assurances from your end.

What on earth is going on with you? Three months of waiting for a simple credit card is absolutely outrageous and completely unacceptable! I not only expect an immediate explanation for this shocking delay, but also a prompt solution. Your bank has failed in every way to meet my expectations of a simple service. I have never experienced anything like this and I will not tolerate it!

I hereby demand that you provide me with a binding response within the next 24 hours, informing me of the exact delivery status and guaranteeing the fastest possible delivery of my card. If this does not happen, I will be forced to take a

In [15]:
print(response.get('summary'))

The customer is extremely frustrated and disappointed with the bank's inability to deliver their credit card after almost three months of waiting. They demand an immediate explanation for the delay and a prompt solution. The customer threatens to terminate their business relationship with the bank and take legal action if the issue is not resolved within 24 hours.


# **Output Parser**

In [20]:
template_string = """You are a master branding consulatant who specializes in naming brands. \
You come up with catchy and memorable brand names.

Take the brand description below delimited by triple backticks and use it to create the name for a brand.

brand description: ```{brand_description}```

then based on the description and you hot new brand name give the brand a score 1-10 for how likely it is to succeed.

{format_instructions}
"""

In [21]:
from langchain.prompts import ChatPromptTemplate
from langchain.output_parsers import ResponseSchema, StructuredOutputParser

In [22]:
prompt = ChatPromptTemplate.from_template(template_string)

In [23]:
brand_name_schema = ResponseSchema(name="brand_name",
                             description="This is the name of the brand")

likelihood_of_success_schema = ResponseSchema(name="likelihood_of_success",
                                      description="This is an integer score between 1-10")

reasoning_schema = ResponseSchema(name="reasoning",
                                    description="This is the reasons for the score")

In [24]:
response_schemas = [brand_name_schema,
                    likelihood_of_success_schema,
                    reasoning_schema]

In [25]:
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
format_instructions = output_parser.get_format_instructions()
print(format_instructions)

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"brand_name": string  // This is the name of the brand
	"likelihood_of_success": string  // This is an integer score between 1-10
	"reasoning": string  // This is the reasons for the score
}
```


In [26]:
messages = prompt.format_messages(brand_description="a cool new sunglasses brand aimed at women",
                                format_instructions=format_instructions)

In [27]:
print(messages[0].content)

You are a master branding consulatant who specializes in naming brands. You come up with catchy and memorable brand names.

Take the brand description below delimited by triple backticks and use it to create the name for a brand.

brand description: ```a cool new sunglasses brand aimed at women```

then based on the description and you hot new brand name give the brand a score 1-10 for how likely it is to succeed.

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"brand_name": string  // This is the name of the brand
	"likelihood_of_success": string  // This is an integer score between 1-10
	"reasoning": string  // This is the reasons for the score
}
```



In [28]:
llm = ChatOpenAI(model='gpt-3.5-turbo', temperature=1.0)
response = llm.invoke(messages)

In [29]:
response = output_parser.parse(response.content)

In [30]:
response.keys()

dict_keys(['brand_name', 'likelihood_of_success', 'reasoning'])

In [31]:
for key in response.keys():
    print(f"{key} : {response[key]}")

brand_name : FemmeShades
likelihood_of_success : 8
reasoning : The name is catchy and appeals directly to the target audience of women looking for stylish sunglasses. It conveys femininity and fashion, which are important aspects for this market.


# **Conversation Summary Buffer Memory**

In [42]:
from langchain.chains import ConversationChain
from langchain.memory import ConversationSummaryBufferMemory

In [43]:
llm2 = ChatOpenAI(model='gpt-3.5-turbo', temperature=1)

In [44]:
memory = ConversationSummaryBufferMemory(llm=llm2, max_token_limit=100)

In [45]:
conversation = ConversationChain(llm = llm2, memory=memory)

In [46]:
conversation.predict(input = 'Give me travel plans to San Francisco')

"Sure! To travel to San Francisco, you have a few options depending on your starting point. If you're flying in, you can check flights to San Francisco International Airport (SFO) or Oakland International Airport (OAK). From the airport, you can take public transportation like BART or a shuttle service to get to the city. If you're driving, be prepared for traffic, especially during peak hours. I can also provide information on popular attractions in San Francisco if you're interested!"

In [47]:
conversation.predict(input = 'Give me some good places to visit while I am there')

"Sure! Some popular attractions in San Francisco include the Golden Gate Bridge, Alcatraz Island, Fisherman's Wharf, Chinatown, and the Painted Ladies houses. You may also want to check out the famous cable cars, explore the diverse neighborhoods like the Mission District and Haight-Ashbury, and enjoy the views from Twin Peaks or Pier 39. There are plenty of museums, parks, and restaurants to explore as well. Let me know if you need more specific recommendations based on your interests!"

In [48]:
conversation.predict(input = 'How can I visit Hollywood while I am there')

' To visit Hollywood while in San Francisco, you can take a flight or drive down to Los Angeles, which is about a 6-hour drive or a 1-hour flight away. Once in Los Angeles, you can explore Hollywood and its famous attractions such as the Hollywood Walk of Fame, TCL Chinese Theatre, the Hollywood Sign, and the Sunset Strip. You can also visit Universal Studios Hollywood or take a tour of celebrity homes. If you prefer not to drive, there are also bus tours available from San Francisco to Los Angeles that include stops in Hollywood. Let me know if you need more information on transportation options or specific attractions in Hollywood!'

In [49]:
print(memory.load_memory_variables([])['history'])

System: The human asks for travel plans to San Francisco, and the AI provides various options depending on the starting point, including flights, public transportation, and driving considerations. The AI also suggests popular attractions like the Golden Gate Bridge, Alcatraz Island, Fisherman's Wharf, and more, as well as exploring diverse neighborhoods and enjoying views from Twin Peaks or Pier 39. The human then asks how to visit Hollywood while in San Francisco, and the AI provides information on traveling to Los Angeles to explore Hollywood's famous attractions, including the Hollywood Walk of Fame, TCL Chinese Theatre, and more.


# **Agent**

In [50]:
!pip install -q wikipedia

In [51]:
from langchain.agents import load_tools, initialize_agent, AgentType
from langchain.memory import ConversationBufferMemory

In [56]:
agentic_memory = ConversationBufferMemory(memory_key = 'sl_ags_memory')
llm = ChatOpenAI(model = 'gpt-3.5-turbo',temperature=1)

In [57]:
tools = load_tools(['llm-math','wikipedia'], llm=llm)

In [58]:
agent = initialize_agent(tools, 
                         llm=llm, 
                         agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
                         memory=agentic_memory,
                         handle_parsing_errors=True,
                         verbose=False
                        )

In [62]:
agent.run(input = 'What is 22+22?')

'44'

In [63]:
agent.run(input = 'What is 5 to the power of 3')

'125'

In [64]:
agent.run(input = 'Who is Richard Feynman')

'Richard Phillips Feynman was an American theoretical physicist known for his work in quantum mechanics, quantum electrodynamics, superfluidity, particle physics, and the development of the parton model. He received the Nobel Prize in Physics in 1965. Feynman also contributed to the field of quantum computing, nanotechnology, and popularized physics through his books and lectures.'